# 지역별 상승하락률예측

In [2]:
import numpy as np
import pandas as pd
import glob

In [3]:
# 푸드 데이터 로드
path_food = "./data/EC_SNS_KFOOD_ATTRACTION_DATA_*.csv"
files_food = glob.glob(path_food)
df_food = [pd.read_csv(f, encoding='utf-8') for f in files_food]
sns_kfood = pd.concat(df_food, ignore_index=True)

# 뷰티 데이터 로드
path_beauty = "./data/EC_SNS_KBEAUTY_ATTRACTION_DATA_*.csv"
files_beauty = glob.glob(path_beauty)
df_beauty = [pd.read_csv(f, encoding='utf-8') for f in files_beauty]
sns_beauty = pd.concat(df_beauty, ignore_index=True)

# 데이터 병합
full_df = pd.concat([sns_kfood, sns_beauty], ignore_index=True)

In [ ]:
# 채널 이름이 체널 전체인 것들만 추출 후 0으로 초기화
full_df = full_df[full_df['CHNNEL_NM'] == '채널전체'].copy()
full_df = full_df.fillna(0)

# 사용할 컬럼 추출
use_cols = ['BASE_YM', 'CTPRVN_NM', 'SIGNGU_NM',
            'BASE_YEAR_ACCMLT_FQ_CO',       # 기준년도누적빈도수
            'TURSM_CSTMR_CO',               # 관광객 수
            'TURSM_SPND_PRICE',             # 관광소비금액 (총합)
            'FRNR_TURSM_SPND_PRICE',        # 외국인관광소비금액
            'AVRG_SCORE_VALUE',             # 평점값
             ] 
df_sub = full_df[use_cols].copy()

# 데이터 형 변환
df_sub['BASE_YM'] = pd.to_datetime(df_sub['BASE_YM'], format='%Y%m')
df_sub = df_sub.fillna(0)

num_cols = [
    'TURSM_CSTMR_CO','TURSM_SPND_PRICE',
    'FRNR_TURSM_SPND_PRICE',
    'AVRG_SCORE_VALUE',
    'BASE_YEAR_ACCMLT_FQ_CO'
]
for c in num_cols:
    df_sub[c] = pd.to_numeric(df_sub[c], errors='coerce')

# 통합 데이터 셋
final_df = df_sub.groupby(['BASE_YM', 'CTPRVN_NM', 'SIGNGU_NM']).agg({
    'TURSM_SPND_PRICE': 'mean',          # 관광소비금액
    'TURSM_CSTMR_CO': 'mean',            # 관광고객수
    'FRNR_TURSM_SPND_PRICE': 'mean',     # 외국인관광소비금액
    'AVRG_SCORE_VALUE': 'mean',          # 평점값
    'BASE_YEAR_ACCMLT_FQ_CO': 'sum',     # 기준년월빈도수
}).reset_index()

# final_df


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# 데이터 로드
df = final_df.copy()

# Unnamed 컬럼 제거
df = df.loc[:, ~df.columns.astype(str).str.startswith("Unnamed")]

# 이상 지역 제거
df = df[df["SIGNGU_NM"].astype(str) != "0"]
df = df[df["SIGNGU_NM"].astype(str) != "KR"]

# 월 컬럼 datetime 변환
df["BASE_YM_DT"] = pd.to_datetime(df["BASE_YM"], errors="coerce")
df = df.dropna(subset=["BASE_YM_DT"])

# 년 월 시 군구로 집계
final_df = (
    df.groupby(["BASE_YM_DT", "CTPRVN_NM", "SIGNGU_NM"], as_index=False)
      .agg({
        #   "TURSM_SPND_PRICE": "mean",          # 소비금액
        #   "TURSM_CSTMR_CO": "mean",            # 관광고객수
          "FRNR_TURSM_SPND_PRICE": "mean",     # 외국인소비
          "BASE_YEAR_ACCMLT_FQ_CO": "sum",     # SNS 누적언급(합계)
      })
)

# 정렬(시계열 필수)
final_df = final_df.sort_values(["CTPRVN_NM", "SIGNGU_NM", "BASE_YM_DT"]).reset_index(drop=True)

In [ ]:
# 시군구별 그룹
group_cols = ["CTPRVN_NM", "SIGNGU_NM"]

# 과거 값 가져오기
# for col in ["TURSM_CSTMR_CO", "TURSM_SPND_PRICE", "BASE_YEAR_ACCMLT_FQ_CO", "FRNR_TURSM_SPND_PRICE"]:
for col in ["BASE_YEAR_ACCMLT_FQ_CO", "FRNR_TURSM_SPND_PRICE"]:
    final_df[f"{col}_L1"] = final_df.groupby(group_cols)[col].shift(1) # 1개월 전
    final_df[f"{col}_L2"] = final_df.groupby(group_cols)[col].shift(2) # 2개월 전

# 전월 대비 성장률 구하기
# for col in ["TURSM_CSTMR_CO", "TURSM_SPND_PRICE", "BASE_YEAR_ACCMLT_FQ_CO", "FRNR_TURSM_SPND_PRICE"]:
for col in ["BASE_YEAR_ACCMLT_FQ_CO", "FRNR_TURSM_SPND_PRICE"]:
    prev = final_df.groupby(group_cols)[col].shift(1)
    final_df[f"{col}_MOM_RATE"] = (final_df[col] - prev) / prev.replace(0, np.nan)

# 최근 3개월(전월 기준) 평균
# for col in ["TURSM_CSTMR_CO", "TURSM_SPND_PRICE", "BASE_YEAR_ACCMLT_FQ_CO", "FRNR_TURSM_SPND_PRICE"]:
for col in ["BASE_YEAR_ACCMLT_FQ_CO", "FRNR_TURSM_SPND_PRICE"]:
    final_df[f"{col}_ROLL3"] = final_df.groupby(group_cols)[col].transform(
        lambda s: s.shift(1).rolling(3, min_periods=1).mean()
    )

# 무한대/결측 정리
final_df = final_df.replace([np.inf, -np.inf], np.nan)

# 다음달 상승/하락 (외국인 소비금액 기준)
# final_df["CSTMR_NEXT"] = final_df.groupby(group_cols)["TURSM_CSTMR_CO"].shift(-1)
final_df["CSTMR_NEXT"] = final_df.groupby(group_cols)["FRNR_TURSM_SPND_PRICE"].shift(-1)

# 마지막 달은 학습에서 제외
final_df["NEXT_GROWTH_LABEL"] = np.where(
    final_df["CSTMR_NEXT"].isna(),
    np.nan,
    # (final_df["CSTMR_NEXT"] > final_df["TURSM_CSTMR_CO"]).astype(int)
    (final_df["CSTMR_NEXT"] > final_df["FRNR_TURSM_SPND_PRICE"]).astype(int)
)

feature_cols = [
    # "TURSM_CSTMR_CO", "TURSM_SPND_PRICE",
    "BASE_YEAR_ACCMLT_FQ_CO",
    "FRNR_TURSM_SPND_PRICE",

    # "TURSM_CSTMR_CO_L1", "TURSM_CSTMR_CO_L2",
    # "TURSM_SPND_PRICE_L1", "TURSM_SPND_PRICE_L2",
    "BASE_YEAR_ACCMLT_FQ_CO_L1", "BASE_YEAR_ACCMLT_FQ_CO_L2",
    "FRNR_TURSM_SPND_PRICE_L1", "FRNR_TURSM_SPND_PRICE_L2",

    # 전월 대비 변화율
    # "TURSM_CSTMR_CO_MOM_RATE", "TURSM_SPND_PRICE_MOM_RATE",
    "BASE_YEAR_ACCMLT_FQ_CO_MOM_RATE",
    "FRNR_TURSM_SPND_PRICE_MOM_RATE",

    # 최근 3개월 추세(롤링)
    # "TURSM_CSTMR_CO_ROLL3", "TURSM_SPND_PRICE_ROLL3",
    "BASE_YEAR_ACCMLT_FQ_CO_ROLL3", "FRNR_TURSM_SPND_PRICE_ROLL3",
]

train_df = final_df.dropna(subset=feature_cols + ["NEXT_GROWTH_LABEL"]).copy()
pred_df  = final_df[final_df["NEXT_GROWTH_LABEL"].isna()].dropna(subset=feature_cols).copy()

X_train = train_df[feature_cols]
y_train = train_df["NEXT_GROWTH_LABEL"]

# 모델 학습
# Logistic Regression
# scaler = StandardScaler()
# X_train_lr = scaler.fit_transform(X_train)

# lr = LogisticRegression(max_iter=5000, class_weight="balanced", random_state=42)
# lr.fit(X_train_lr, y_train)

# RandomForest로 확률 예측
rf = RandomForestClassifier(
    n_estimators=1200,
    min_samples_leaf=2,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

# 예측
pred_df["proba_up"] = rf.predict_proba(pred_df[feature_cols])[:, 1]
pred_df["proba_down"] = 1 - pred_df["proba_up"]
pred_df["pred_label"] = (pred_df["proba_up"] >= 0.5).astype(int)

pred_df.sort_values("proba_up", ascending=False)

top10 = pred_df.sort_values("proba_up", ascending=False).tail(10)

for _, r in top10.iterrows():
    label = 1 if r["proba_up"] >= 0.5 else 0
    print(f"지역 : {r['CTPRVN_NM']} {r['SIGNGU_NM']}")
    print(f"예측 결과: 인기 {'상승' if label==1 else '하락'}")
    print(f"상승 확률: {r['proba_up']:.2f}")
    print(f"하락 확률: {r['proba_down']:.2f}")
    print("-"*40)


지역 : 부산광역시 서구
예측 결과: 인기 하락
상승 확률: 0.31
하락 확률: 0.69
----------------------------------------
지역 : 대구광역시 중구
예측 결과: 인기 하락
상승 확률: 0.28
하락 확률: 0.72
----------------------------------------
지역 : 경기도 용인시 처인구
예측 결과: 인기 하락
상승 확률: 0.26
하락 확률: 0.74
----------------------------------------
지역 : 경기도 수원시
예측 결과: 인기 하락
상승 확률: 0.26
하락 확률: 0.74
----------------------------------------
지역 : 서울특별시 구로구
예측 결과: 인기 하락
상승 확률: 0.24
하락 확률: 0.76
----------------------------------------
지역 : 대전광역시 서구
예측 결과: 인기 하락
상승 확률: 0.23
하락 확률: 0.77
----------------------------------------
지역 : 경기도 고양시
예측 결과: 인기 하락
상승 확률: 0.17
하락 확률: 0.83
----------------------------------------
지역 : 경기도 가평군
예측 결과: 인기 하락
상승 확률: 0.16
하락 확률: 0.84
----------------------------------------
지역 : 서울특별시 성북구
예측 결과: 인기 하락
상승 확률: 0.13
하락 확률: 0.87
----------------------------------------
지역 : 부산광역시 수영구
예측 결과: 인기 하락
상승 확률: 0.10
하락 확률: 0.90
----------------------------------------


---

In [7]:
pred_df[['CTPRVN_NM', 'SIGNGU_NM', 'proba_up','proba_down']]

,CTPRVN_NM,SIGNGU_NM,proba_up,proba_down
11,강원특별자치도,강릉시,0.483746,0.516254
18,강원특별자치도,속초시,0.678934,0.321066
35,강원특별자치도,춘천시,0.584626,0.415374
40,강원특별자치도,홍천군,0.528561,0.471439
47,경기도,가평군,0.162643,0.837357
...,...,...,...,...
834,충청남도,천안시,0.525365,0.474635
839,충청남도,천안시 동남구,0.798582,0.201418
843,충청남도,청양군,0.851509,0.148491
849,충청북도,청주시,0.430241,0.569759


In [8]:
final_df["CSTMR_NEXT"]

0            NaN
1       446404.0
2       578600.0
3       765899.0
4       684540.0
         ...    
851      47223.0
852     719580.0
853    1197637.0
854    1326392.0
855          NaN
Name: CSTMR_NEXT, Length: 856, dtype: float64